# Unsupervised Music Genre Discovery Using Audio Feature Learning

This notebook provides an interactive analysis of music genre discovery using clustering algorithms on Spotify audio features.

## Overview
- **Dataset**: Spotify audio features (~170K tracks)
- **Algorithms**: K-Means, Spectral, DBSCAN, GMM
- **Evaluation**: Internal and external clustering metrics
- **Visualization**: Comprehensive plots and analysis

In [ ]:
# Import required libraries
import warnings
warnings.filterwarnings('ignore')

from music_genre_analysis import MusicGenreAnalyzer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Libraries loaded successfully!")

## 1. Initialize Analyzer and Load Data

In [ ]:
# Initialize the analyzer
analyzer = MusicGenreAnalyzer()

# Load and preprocess data
print("Loading and preprocessing data...")
data = analyzer.load_and_preprocess_data()

# Display basic information
print(f"\nDataset Shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
print(f"Year Range: {data['year'].min()} - {data['year'].max()}")

# Show first few rows
data.head()

## 2. Exploratory Data Analysis

In [ ]:
# Perform comprehensive EDA
print("Performing Exploratory Data Analysis...")
eda_results = analyzer.exploratory_data_analysis()

# Display basic statistics
print("\nBasic Statistics:")
eda_results['basic_stats']

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(analyzer.audio_features):
    if i < len(axes):
        data[feature].hist(bins=50, ax=axes[i], alpha=0.7, color='skyblue', edgecolor='black')
        axes[i].set_title(f'Distribution of {feature.capitalize()}', fontsize=12, fontweight='bold')
        axes[i].set_xlabel(feature.capitalize())
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Audio Feature Distributions', fontsize=16, fontweight='bold', y=1.02)
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 10))
correlation_matrix = data[analyzer.all_features].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdBu_r',
           center=0, square=True, fmt='.2f', 
           cbar_kws={"shrink": .8, "label": "Correlation Coefficient"})

plt.title('Audio Features Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Display highly correlated pairs
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:  # Threshold for high correlation
            corr_pairs.append({
                'Feature 1': correlation_matrix.columns[i],
                'Feature 2': correlation_matrix.columns[j],
                'Correlation': corr_val
            })

if corr_pairs:
    corr_df = pd.DataFrame(corr_pairs).sort_values('Correlation', key=abs, ascending=False)
    print("\nHighly Correlated Feature Pairs (|correlation| > 0.5):")
    print(corr_df)

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(analyzer.audio_features):
    if i < len(axes):
        bp = axes[i].boxplot(data[feature], patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        bp['boxes'][0].set_alpha(0.7)
        axes[i].set_title(f'Box Plot: {feature.capitalize()}', fontsize=12, fontweight='bold')
        axes[i].set_ylabel(feature.capitalize())
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Box Plots for Outlier Detection', fontsize=16, fontweight='bold', y=1.02)
plt.show()

## 3. Feature Engineering and Preparation

In [ ]:
# Prepare features for clustering
print("Preparing features for clustering...")
processed_features = analyzer.prepare_features()

print(f"Original features shape: {analyzer.features.shape}")
print(f"Processed features shape: {processed_features.shape}")
print(f"Explained variance ratio (cumulative): {analyzer.pca.explained_variance_ratio_.sum():.3f}")

# Visualize PCA components
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(1, len(analyzer.pca.explained_variance_ratio_) + 1), 
        analyzer.pca.explained_variance_ratio_, alpha=0.7, color='steelblue')
plt.title('PCA Explained Variance Ratio', fontweight='bold')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
cumsum = np.cumsum(analyzer.pca.explained_variance_ratio_)
plt.plot(range(1, len(cumsum) + 1), cumsum, marker='o', linewidth=2, color='darkred')
plt.title('Cumulative Explained Variance', fontweight='bold')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.95, color='green', linestyle='--', alpha=0.7, label='95% threshold')
plt.legend()

plt.tight_layout()
plt.show()

## 4. Clustering Analysis

In [ ]:
# Perform clustering with all algorithms
n_clusters = 10  # Adjust as needed
print(f"Performing clustering with {n_clusters} clusters...")

clustering_results = analyzer.perform_clustering(n_clusters=n_clusters)

# Display clustering summary
print("\nClustering Results Summary:")
print("=" * 50)
for name, results in clustering_results.items():
    if 'error' in results:
        print(f"{name}: ERROR - {results['error']}")
    else:
        n_clusters_found = results['n_clusters']
        print(f"{name}: {n_clusters_found} clusters found")

In [ ]:
# Visualize clustering results in PCA space
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

valid_results = [(name, results) for name, results in clustering_results.items() 
                if results['labels'] is not None and 'error' not in results]

for i, (algorithm_name, results) in enumerate(valid_results[:6]):
    if i >= len(axes):
        break
        
    ax = axes[i]
    labels = results['labels']
    
    # Create scatter plot
    scatter = ax.scatter(processed_features[:, 0], processed_features[:, 1],
                        c=labels, cmap='tab20', alpha=0.6, s=20, edgecolors='black', linewidth=0.1)
    
    ax.set_title(f'{algorithm_name}\n({results["n_clusters"]} clusters)', 
                fontsize=12, fontweight='bold')
    ax.set_xlabel('First Principal Component')
    ax.set_ylabel('Second Principal Component')
    ax.grid(True, alpha=0.3)
    
    # Add colorbar for non-DBSCAN algorithms
    if algorithm_name != 'DBSCAN' or -1 not in labels:
        plt.colorbar(scatter, ax=ax, label='Cluster ID')

# Remove empty subplots
for j in range(len(valid_results), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle('Clustering Results Visualization (PCA Space)', fontsize=16, fontweight='bold', y=1.02)
plt.show()

## 5. Clustering Evaluation

In [ ]:
# Evaluate clustering performance
print("Evaluating clustering results...")
evaluation_results = analyzer.evaluate_clustering()

if not evaluation_results.empty:
    # Display results table
    print("\nClustering Evaluation Results:")
    print("=" * 100)
    print(evaluation_results.to_string(index=False))
    
    # Find best performing algorithms
    print("\n" + "=" * 50)
    print("PERFORMANCE HIGHLIGHTS")
    print("=" * 50)
    
    best_silhouette = evaluation_results.loc[evaluation_results['Silhouette_Score'].idxmax()]
    print(f"Best Silhouette Score: {best_silhouette['Algorithm']} ({best_silhouette['Silhouette_Score']:.4f})")
    
    best_dbi = evaluation_results.loc[evaluation_results['Davies_Bouldin_Index'].idxmin()]
    print(f"Best Davies-Bouldin Index: {best_dbi['Algorithm']} ({best_dbi['Davies_Bouldin_Index']:.4f})")
    
    best_chi = evaluation_results.loc[evaluation_results['Calinski_Harabasz_Index'].idxmax()]
    print(f"Best Calinski-Harabasz Index: {best_chi['Algorithm']} ({best_chi['Calinski_Harabasz_Index']:.2f})")
else:
    print("No evaluation results available.")

In [ ]:
# Visualize evaluation metrics
if not evaluation_results.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Internal metrics
    metrics_to_plot = [
        ('Silhouette_Score', 'Silhouette Score', 'Higher is Better'),
        ('Davies_Bouldin_Index', 'Davies-Bouldin Index', 'Lower is Better'),
        ('Calinski_Harabasz_Index', 'Calinski-Harabasz Index', 'Higher is Better'),
        ('Normalized_Mutual_Info', 'Normalized Mutual Information', 'Higher is Better')
    ]
    
    for i, (metric, title, note) in enumerate(metrics_to_plot):
        ax = axes[i//2, i%2]
        
        bars = ax.bar(evaluation_results['Algorithm'], evaluation_results[metric], 
                     alpha=0.7, color='steelblue', edgecolor='black')
        
        ax.set_title(f'{title}\n({note})', fontweight='bold')
        ax.set_xlabel('Algorithm')
        ax.set_ylabel(title)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, value in zip(bars, evaluation_results[metric]):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + ax.get_ylim()[1]*0.01,
                   f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Comprehensive metrics heatmap
    plt.figure(figsize=(12, 8))
    
    # Select numeric metrics for heatmap
    numeric_cols = ['Silhouette_Score', 'Davies_Bouldin_Index', 'Calinski_Harabasz_Index',
                   'Adjusted_Rand_Index', 'Normalized_Mutual_Info', 'V_Measure', 'Purity']
    
    # Create normalized heatmap data for better visualization
    heatmap_data = evaluation_results[numeric_cols].T
    heatmap_data.columns = evaluation_results['Algorithm']
    
    # Normalize metrics (invert Davies-Bouldin since lower is better)
    normalized_data = heatmap_data.copy()
    for col in normalized_data.columns:
        if 'Davies_Bouldin' in normalized_data.index:
            normalized_data.loc['Davies_Bouldin_Index', col] = 1 - normalized_data.loc['Davies_Bouldin_Index', col]
    
    sns.heatmap(heatmap_data, annot=True, cmap='RdYlBu_r', center=0,
               fmt='.3f', cbar_kws={'label': 'Metric Value'})
    
    plt.title('Clustering Evaluation Metrics Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Clustering Algorithm')
    plt.ylabel('Evaluation Metric')
    plt.tight_layout()
    plt.show()

## 6. Cluster Analysis and Interpretation

In [ ]:
# Analyze cluster characteristics for best performing algorithm
if not evaluation_results.empty:
    best_algorithm = evaluation_results.loc[evaluation_results['Silhouette_Score'].idxmax(), 'Algorithm']
    best_labels = clustering_results[best_algorithm]['labels']
    
    print(f"Analyzing cluster characteristics for: {best_algorithm}")
    print("=" * 60)
    
    # Add cluster labels to original data
    data_with_clusters = data.copy()
    data_with_clusters['cluster'] = best_labels
    
    # Calculate cluster statistics
    cluster_stats = data_with_clusters.groupby('cluster')[analyzer.audio_features].agg(['mean', 'std']).round(3)
    
    # Display cluster sizes
    cluster_sizes = data_with_clusters['cluster'].value_counts().sort_index()
    print("\nCluster Sizes:")
    for cluster_id, size in cluster_sizes.items():
        if cluster_id != -1:  # Exclude noise points for DBSCAN
            percentage = (size / len(data_with_clusters)) * 100
            print(f"Cluster {cluster_id}: {size:,} tracks ({percentage:.1f}%)")
    
    # Visualize cluster characteristics
    plt.figure(figsize=(15, 10))
    
    # Select key features for radar chart
    key_features = ['danceability', 'energy', 'valence', 'acousticness', 'instrumentalness']
    
    # Create subplot for each cluster (limit to first 6 clusters)
    valid_clusters = [c for c in cluster_sizes.index if c != -1][:6]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for i, cluster_id in enumerate(valid_clusters):
        if i >= len(axes):
            break
            
        ax = axes[i]
        cluster_data = data_with_clusters[data_with_clusters['cluster'] == cluster_id]
        
        # Create bar plot for cluster characteristics
        feature_means = cluster_data[key_features].mean()
        bars = ax.bar(key_features, feature_means, alpha=0.7, color=f'C{i}', edgecolor='black')
        
        ax.set_title(f'Cluster {cluster_id} Characteristics\n({len(cluster_data):,} tracks)', 
                    fontweight='bold')
        ax.set_ylabel('Average Feature Value')
        ax.set_ylim(0, 1)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3)
        
        # Add value labels
        for bar, value in zip(bars, feature_means):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                   f'{value:.2f}', ha='center', va='bottom', fontsize=9)
    
    # Remove empty subplots
    for j in range(len(valid_clusters), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.suptitle(f'{best_algorithm} - Cluster Characteristics Analysis', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.show()

In [ ]:
# Temporal analysis of clusters
if not evaluation_results.empty and 'data_with_clusters' in locals():
    # Analyze cluster evolution over decades
    decade_cluster_analysis = data_with_clusters.groupby(['decade', 'cluster']).size().unstack(fill_value=0)
    
    # Calculate proportions
    decade_proportions = decade_cluster_analysis.div(decade_cluster_analysis.sum(axis=1), axis=0)
    
    plt.figure(figsize=(14, 8))
    
    # Stacked area plot
    valid_clusters_for_plot = [c for c in decade_proportions.columns if c != -1][:8]  # Limit colors
    
    plt.stackplot(decade_proportions.index, 
                 *[decade_proportions[cluster] for cluster in valid_clusters_for_plot],
                 labels=[f'Cluster {c}' for c in valid_clusters_for_plot],
                 alpha=0.8)
    
    plt.title('Evolution of Music Clusters Over Decades', fontsize=16, fontweight='bold')
    plt.xlabel('Decade')
    plt.ylabel('Proportion of Tracks')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Show most representative tracks for each cluster
    print("\nMost Representative Tracks by Cluster:")
    print("=" * 80)
    
    for cluster_id in valid_clusters[:3]:  # Show first 3 clusters
        cluster_tracks = data_with_clusters[data_with_clusters['cluster'] == cluster_id]
        
        # Sample popular tracks from the cluster
        representative_tracks = cluster_tracks.nlargest(5, 'popularity')[['name', 'artists', 'year', 'popularity']]
        
        print(f"\nCluster {cluster_id} - Top 5 Popular Tracks:")
        print("-" * 50)
        for idx, track in representative_tracks.iterrows():
            print(f"• {track['name']} by {track['artists']} ({track['year']}) - Popularity: {track['popularity']}")

## 7. Advanced Analysis and Insights

In [ ]:
# Feature importance analysis
if hasattr(analyzer, 'pca'):
    # Get PCA component loadings
    feature_names = analyzer.all_features
    components = analyzer.pca.components_
    
    # Create feature importance matrix
    feature_importance = pd.DataFrame(
        components[:5].T,  # First 5 components
        index=feature_names,
        columns=[f'PC{i+1}' for i in range(5)]
    )
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(feature_importance, annot=True, cmap='RdBu_r', center=0,
               fmt='.3f', cbar_kws={'label': 'Component Loading'})
    
    plt.title('Feature Loadings on Principal Components', fontsize=16, fontweight='bold')
    plt.xlabel('Principal Component')
    plt.ylabel('Audio Feature')
    plt.tight_layout()
    plt.show()
    
    # Show top contributing features for each component
    print("\nTop Contributing Features for Each Principal Component:")
    print("=" * 70)
    
    for i in range(3):  # First 3 components
        pc_name = f'PC{i+1}'
        top_features = feature_importance[pc_name].abs().nlargest(5)
        
        print(f"\n{pc_name} (Explains {analyzer.pca.explained_variance_ratio_[i]:.1%} of variance):")
        print("-" * 40)
        
        for feature, loading in top_features.items():
            direction = "positively" if feature_importance.loc[feature, pc_name] > 0 else "negatively"
            print(f"• {feature}: {loading:.3f} ({direction} correlated)")

## 8. Summary and Conclusions

In [ ]:
# Generate final summary
print("\n" + "=" * 80)
print("ANALYSIS SUMMARY AND CONCLUSIONS")
print("=" * 80)

print(f"\n📊 Dataset Overview:")
print(f"   • Total tracks analyzed: {len(data):,}")
print(f"   • Audio features: {len(analyzer.audio_features)}")
print(f"   • Year range: {data['year'].min()} - {data['year'].max()}")
print(f"   • Dimensionality reduction: {analyzer.features.shape[1]} → {analyzer.processed_data.shape[1]} features")

if not evaluation_results.empty:
    print(f"\n🎯 Clustering Results:")
    print(f"   • Algorithms tested: {len(evaluation_results)}")
    
    best_silhouette_row = evaluation_results.loc[evaluation_results['Silhouette_Score'].idxmax()]
    print(f"   • Best overall performance: {best_silhouette_row['Algorithm']}")
    print(f"     - Silhouette Score: {best_silhouette_row['Silhouette_Score']:.4f}")
    print(f"     - Davies-Bouldin Index: {best_silhouette_row['Davies_Bouldin_Index']:.4f}")
    print(f"     - Clusters found: {best_silhouette_row['N_Clusters']}")
    
    print(f"\n📈 Key Findings:")
    
    # Algorithm performance ranking
    silhouette_ranking = evaluation_results.sort_values('Silhouette_Score', ascending=False)
    print(f"   • Algorithm ranking (by Silhouette Score):")
    for i, (_, row) in enumerate(silhouette_ranking.iterrows(), 1):
        print(f"     {i}. {row['Algorithm']}: {row['Silhouette_Score']:.4f}")
    
    # Feature insights
    high_corr_features = []
    corr_matrix = data[analyzer.audio_features].corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.6:
                high_corr_features.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    
    if high_corr_features:
        print(f"\n   • Highly correlated features (|r| > 0.6):")
        for feat1, feat2, corr in sorted(high_corr_features, key=lambda x: abs(x[2]), reverse=True)[:3]:
            print(f"     - {feat1} ↔ {feat2}: {corr:.3f}")

print(f"\n💡 Recommendations:")
print(f"   • Use {evaluation_results.loc[evaluation_results['Silhouette_Score'].idxmax(), 'Algorithm']} for production clustering")
print(f"   • Consider ensemble methods combining top-performing algorithms")
print(f"   • Explore genre-specific feature engineering")
print(f"   • Implement hierarchical clustering for genre taxonomies")

print(f"\n📁 Generated Outputs:")
print(f"   • All visualizations and analysis plots")
print(f"   • Processed feature matrices and cluster assignments")
print(f"   • Comprehensive evaluation metrics")
print(f"   • Interactive analysis results")

print("\n" + "=" * 80)
print("Analysis completed successfully! 🎵📊")
print("=" * 80)

## 9. Export Results

In [ ]:
# Save results and generate report
print("Saving results and generating report...")

# Save all results
analyzer.save_results("notebook_results/")

# Generate HTML report
report_path = analyzer.generate_report("notebook_analysis_report.html")

print(f"\n✅ Results saved successfully!")
print(f"📊 HTML Report: {report_path}")
print(f"📁 Numerical Results: notebook_results/")
print(f"\nYou can now view the comprehensive HTML report for detailed findings.")